# Exercise for Unit 6

**Name:** MYRRHEA BELLE B. JUNSAY

**Date:** NOVEMBER 19, 2025  

**Year and Section:** BSCS 3A AI

In [3]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [4]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = '../../muffin_vs_chihuahua/train'
test_dir = '../../muffin_vs_chihuahua/test'    # e.g. './muffin-vs-chihuahua/test'

In [5]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [6]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [13]:
# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(0.0005),
                   input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(0.0005)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.3),

    layers.Conv2D(128, (3, 3), activation='relu',
                   kernel_regularizer=regularizers.l2(0.0005)),
    layers.MaxPooling2D(2, 2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\miray\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Configure the model
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [68]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 116s 953ms/step - accuracy: 0.6064 - loss: 0.7141 - val_accuracy: 0.7725 - val_loss: 0.5336
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.7836 - loss: 0.5144 - val_accuracy: 0.8074 - val_loss: 0.4772
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 248s 2s/step - accuracy: 0.7988 - loss: 0.4575 - val_accuracy: 0.8466 - val_loss: 0.4001
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 453s 4s/step - accuracy: 0.8329 - loss: 0.4245 - val_accuracy: 0.8508 - val_loss: 0.4079
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.8542 - loss: 0.3842 - val_accuracy: 0.8582 - val_loss: 0.3731
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.8552 - loss: 0.3875 - val_accuracy: 0.8635 - val_loss: 0.3861
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.8550 - loss: 0.3616 - val_accuracy: 0.9058 - val_loss: 0.2696
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.8569 - loss: 0.3624 - val_a

In [69]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy of Improved Model: {test_acc:.4f}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 16s 426ms/step - accuracy: 0.8637 - loss: 0.3556
Test Accuracy of Improved Model: 0.8885


In [70]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [71]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image
import warnings
warnings.filterwarnings('ignore')

# Load the model once
loaded_model = tf.keras.models.load_model('exercise_6_trained_model_improved.h5')

def predict_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    pred = loaded_model.predict(img_array, verbose=0)[0,0]
    
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    confidence = pred if label == "Chihuahua" else 1 - pred
    
    print(f"\nImage: {img_path}")
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

In [72]:
# Example usage:
predict_image("../../run/run_4.jpg")
predict_image("../../run/run_6.jpg")


Image: ../../run/run_4.jpg
Prediction: Muffin (confidence: 0.68)

Image: ../../run/run_6.jpg
Prediction: Chihuahua (confidence: 0.89)


## 3. Training on Dogs vs. Cats Dataset

In [10]:
custom_train_dir = '../../dog_vs_cat/training_set/training_set'
custom_test_dir = '../../dog_vs_cat/test_set/test_set'

In [11]:
train_generator_custom = train_datagen.flow_from_directory(
    custom_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator_custom = train_datagen.flow_from_directory(
    custom_train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_generator_custom = test_datagen.flow_from_directory(
    custom_test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [14]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
print("\nRetraining the CNN model on the Dogs vs Cats dataset")
history_custom = model.fit(
    train_generator_custom,
    epochs=10,
    validation_data=val_generator_custom
)

# EVALUATE THE NEWLY TRAINED MODEL
print("\n--- Evaluating the Custom Dataset Model ---")
custom_loss, custom_acc = model.evaluate(test_generator_custom)
print(f"Test Accuracy on Cats vs. Dogs: {custom_acc:.4f}")

# SAVE THE MODEL WITH THE CUSTOM NAME
model.save('exercise_6_custom_Junsay.h5')
print("\nCustom model saved as 'exercise_6_custom_Junsay.h5'")


Retraining the CNN model on the Dogs vs Cats dataset
Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 552s 3s/step - accuracy: 0.7310 - loss: 0.5553 - val_accuracy: 0.7071 - val_loss: 0.5721
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 707s 3s/step - accuracy: 0.7263 - loss: 0.5517 - val_accuracy: 0.7452 - val_loss: 0.5267
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 169s 837ms/step - accuracy: 0.7352 - loss: 0.5409 - val_accuracy: 0.7171 - val_loss: 0.5520
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 149s 739ms/step - accuracy: 0.7407 - loss: 0.5348 - val_accuracy: 0.7552 - val_loss: 0.5201
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 151s 750ms/step - accuracy: 0.7442 - loss: 0.5302 - val_accuracy: 0.7320 - val_loss: 0.5249
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 150s 743ms/step - accuracy: 0.7452 - loss: 0.5265 - val_accuracy: 0.7295 - val_loss: 0.5448
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 155s 767ms/step - accuracy: 0.7565 - loss: 0.5182 - val_accuracy: 0.7583 - val_loss: 0.5182
Epoch 8/10
201/201 ━━━━━━━━━

Test Accuracy on Cats vs. Dogs: 0.7865

Custom model saved as 'exercise_6_custom_Junsay.h5'


## 4. USING A PRE-TRAINED ARCHITECTURE (VGG16)

In [17]:
print("\nTransfer Learning with VGG16")

from tensorflow.keras.applications import VGG16
from tensorflow.keras import optimizers

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3)) # Match our IMG_SIZE

conv_base.trainable = False

# 3. ADD OUR OWN CLASSIFIER ON TOP
vgg_model = models.Sequential([
    conv_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') # Our final output layer
])


vgg_model.compile(optimizer=optimizers.Adam(learning_rate=1e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

vgg_model.summary()

print("\n--- Training the new classifier head with VGG16 base ---")
history_vgg = vgg_model.fit(
    train_generator, # Using the original muffin/chihuahua generator
    epochs=10,
    validation_data=val_generator
)


print("\n--- Evaluating the VGG16-based Model ---")
vgg_loss, vgg_acc = vgg_model.evaluate(test_generator)
print(f"Final Test Accuracy with VGG16: {vgg_acc:.4f}")


Transfer Learning with VGG16
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 55s 1us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,812,353 (64.13 MB)

 Trainable params: 2,097,665 (8.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)


--- Training the new classifier head with VGG16 base ---
Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 535s 4s/step - accuracy: 0.6562 - loss: 0.6129 - val_accuracy: 0.8910 - val_loss: 0.3287
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 453s 4s/step - accuracy: 0.8636 - loss: 0.3378 - val_accuracy: 0.9048 - val_loss: 0.2533
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 434s 4s/step - accuracy: 0.8803 - loss: 0.2699 - val_accuracy: 0.9164 - val_loss: 0.2209
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 448s 4s/step - accuracy: 0.9199 - loss: 0.2160 - val_accuracy: 0.9312 - val_loss: 0.2006
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 525s 4s/step - accuracy: 0.9210 - loss: 0.2057 - val_accuracy: 0.9228 - val_loss: 0.1813
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 397s 3s/step - accuracy: 0.9344 - loss: 0.1830 - val_accuracy: 0.9312 - val_loss: 0.1770
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 394s 3s/step - accuracy: 0.9337 - loss: 0.1708 - val_accuracy: 0.9397 - val_loss: 0.1613
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━